In [2]:
from pyspark import SparkConf, SparkContext
from operator import add
import pyspark.sql.functions as f
from pyspark.sql.functions import desc

## mapper1
Split the original file by ASCII 9 (long space)

In [3]:
def mapper1(line):
    fromNode, toNode = line.split(chr(9))
    maplist = []
    maplist.append((fromNode, toNode))
    return maplist

## reducer1
while initialize the rankings, merge the same node without changing their rankings 1/N

In [4]:
def reducer1(x, y):
    return y

## reducer2
add all contributions for each nodes

In [5]:
def reducer2(x, y):
    return x + y

## mapper3
calculate the contribution, since I add self-to-self route, I separate it from the original ones, for self-to-self the contribution is 0, for original ones the contribution is rank/N

In [6]:
def mapper3(self, urls, rank):
    num_urls = len(urls)
    for url in urls:
        if self == url:
            yield (url, 0)
        else:
            yield (url, rank / (num_urls-1))

## mapper4
renormalize the map by adding S, S is calculated according to the formula on the HW2 pdf

In [7]:
def mapper4(line1, line2, S):
    maplist = []
    maplist.append((line1, line2 + S))
    return maplist

## mapper5
to split the original data in to in-node and out-node, this mapper is for in-node

In [17]:
def mapper5(line):
    maplist = []
    maplist.append(line[0])
    return maplist

## mapper5
to split the original data in to in-node and out-node, this mapper is for out-node

In [19]:
def mapper6(line):
    maplist = []
    maplist.append(line[1])
    return maplist

In [23]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("wordcount")
sc = SparkContext(conf=conf)
links = sc.textFile("data2.txt").flatMap(mapper1)

urlNum1 = links.flatMap(mapper5)
urlNum2 = links.flatMap(mapper6)
urlNum = urlNum1.union(urlNum2).distinct().count()
#print(urlNum)

ranks1 = links.map(lambda line : (line[0], 1/int(urlNum))).reduceByKey(reducer1)
ranks2 = links.map(lambda line : (line[1], 1/int(urlNum))).reduceByKey(reducer1)
ranks = ranks1.union(ranks2).distinct().sortByKey()
#print(ranks.collect())

subnode_links = ranks.map(lambda line: (line[0], line[0]))
join_links = links.union(subnode_links).distinct().sortByKey()
#print(subnode_links.collect())

lines = join_links.distinct().groupByKey().mapValues(lambda x: list(x)).cache()
contribs = lines.join(ranks)
#print(contribs.collect())

for i in range(20):
    contribs = lines.join(ranks).flatMap(lambda x: mapper3(x[0], x[1][0], x[1][1]))
    #print(contribs.collect())
    ranks = contribs.reduceByKey(reducer2).mapValues(lambda rank: rank*0.8 + 0.2/int(urlNum))
    #print(ranks.collect())
    sigma = ranks.map(lambda x : x[1]).sum()
    S = (1 - sigma)/int(urlNum)
    #print(S)
    ranks = ranks.flatMap(lambda x: mapper4(x[0], x[1], S)).sortBy(lambda a: -a[1])
    #print(newranks.collect())

count = 0
for (link, rank) in ranks.collect():
    if(count < 10):
        #print("%s	%s" % (link, rank))
        print("%s	%.06f" % (link, rank))
        count += 1
    elif(count == 10):
        break

'''
outputFile = open("Outputfile.txt", "w")
for (link, rank) in ranks.collect():
    result = str(link) + chr(9) + str(format(rank,'.03f'))
    outputFile.write("%s" % (result) + "\n")
    #print("%s has rank: %.03f" % (link, rank))
outputFile.close()
'''

sc.stop()

1056	0.000632
1054	0.000629
1536	0.000524
171	0.000512
453	0.000496
407	0.000485
263	0.000480
4664	0.000470
261	0.000463
410	0.000462
